In [ ]:
from utils.before_after import *
from collections import Counter, defaultdict
import spacy
from utils import explacy # https://github.com/tylerneylon/explacy
# explacy.print_parse_info(nlp, line)

In [ ]:
nlp = spacy.load('en_core_web_lg') # ('en')

## Parse BNC 所有句子，再取出有 discuss
* read all lines
* strip() and lower()
* filter out target sentences
* ** Use list() to keep filter generator **

In [ ]:
TARGET_WORD = 'discuss'

In [ ]:
# bnc_remains = filter(lambda line: any([tk.lemma_ == TARGET_WORD for tk in line]), bnc_all)

In [5]:
def clean_data(file, reserved=None):
    lines = map(lambda line: line.strip().lower(), open(file, 'r', encoding='utf8'))
    remains = filter(lambda line: reserved in line, lines) if reserved else lines
    return list(remains)

bnc_temp = clean_data('../dataset/bnc.txt', TARGET_WORD)
bnc_remains = list(map(lambda line: nlp(line), bnc_temp))

## 從 EF 中取得有 discuss 的句子，並且 parse

In [ ]:
# ef_temp = clean_data('../dataset/efcamp/ef.diff.simplize.despace.txt', 'discuss')
# ef_remains = []
# for index, line in enumerate(ef_temp):
#     tokens = line.split(' ')
    
#     aft_sent = nlp(' '.join(to_after(tokens)))
#     bef_sent = nlp(' '.join(to_before(tokens)))
#     has_edit = any([tk[:2] in ['[-', '{+'] and TARGET_WORD in tokens[i-1] 
#                     for i, tk in enumerate(tokens)]) 
    
#     ef_remains.append({
#         'origin': line,
#         'bef_sent': bef_sent,
#         'aft_sent': aft_sent, 
#         'has_edit': has_edit
#     })

## 用 dependency 抓 pattern
#### 第一層 dependency
dobj, prep, nsubj, nsubjpass, ccomp, xcomp, csubj, csubjpass, prt, acomp, oprd

#### 第二層 dependency
prep -> pobj, pcomp


In [ ]:
if __name__ == '__main__':
    # Head word only VERBS
    # headword, patterns, dep, ngrams
    patterns = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: [])))
    sents = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: []))) # for debug

    for line in bnc_remains:
        for tk in line:
            if tk.tag_ in VERBS: 
                ptn, ngram = dep_to_pattern(tk)

                patterns[tk.lemma_][tk.dep_][ptn].append(ngram)
                sents[tk.lemma_][tk.dep_][ptn].append(tk.doc.text)

In [ ]:
# patterns['discuss']
# sents['conj']['V-ed']

In [26]:

explacy.print_parse_info(nlp, 'I like the girl who takes an umbrella .')
explacy.print_parse_info(nlp, 'I would like to discuss the problem you like on Monday.')

Dep tree      Token    Dep type Lemma    Tag Part of Sp
───────────── ──────── ──────── ──────── ─── ──────────
          ┌─> I        nsubj    -PRON-   PRP PRON      
┌┬────────┴── like     ROOT     like     VBP VERB      
││        ┌─> the      det      the      DT  DET       
│└─>┌─────┴── girl     dobj     girl     NN  NOUN      
│   │     ┌─> who      nsubj    who      WP  NOUN      
│   └─>┌──┴── takes    relcl    take     VBZ VERB      
│      │  ┌─> an       det      an       DT  DET       
│      └─>└── umbrella dobj     umbrella NN  NOUN      
└───────────> .        punct    .        .   PUNCT     
Dep tree       Token   Dep type Lemma   Tag Part of Sp
────────────── ─────── ──────── ─────── ─── ──────────
          ┌──> I       nsubj    -PRON-  PRP PRON      
          │┌─> would   aux      would   MD  VERB      
┌┬────────┴┴── like    ROOT     like    VB  VERB      
││         ┌─> to      aux      to      TO  PART      
│└─>┌┬─────┴── discuss xcomp    discuss VB  VERB      